In [58]:
from tqdm.auto import tqdm
import numpy as np
import json, glob, os
import pandas as pd
import re, string
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import math

In [59]:
def read_(path):
    with open(path, "r") as f:
        return f.readlines()

In [60]:
lex = read_('data/english-hindi-dictionary.txt')
eng = read_('data/english.txt')
hindi = read_('data/hindi.txt')
out = read_('data/BingLiu.csv')

In [61]:
lex = { i.split('|||')[0].strip():i.split('|||')[-1].strip() for i in lex if not re.search('[a-zA-Z]', i.split('|||')[-1].strip()) }
# lex

In [62]:
# # saving for glove 
# a = ''
# for i in eng:
#     a+=i.strip().lower() + ' '
    
# with open('GloVe/eng.txt', 'w') as f:
#     f.write(a)

## L1 

In [63]:
# en_word1, hindi_word1, polarity

In [64]:
out = [[j.strip() for j in i.split('\t')] for i in out]

In [65]:
def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 

In [66]:
L1 = ''
for i in tqdm(out):
    try: L1 += f"{remove_punctuation(i[0])},{remove_punctuation(lex[i[0]])},{i[-1]}\n"
    except: pass #L1 += f"{i[0]},{None},{i[-1]}\n"


In [69]:
with open('save/L1.txt', 'w') as f:
    f.write(L1)

In [70]:
print(len(L1.split('\n')))

2115


## Embeddings

In [71]:
# eng_ = [[j.lower() for j in i.split()]for i in eng]
# hindi_ = [[j.lower() for j in i.split()]for i in hindi]

# w2v_eng = Word2Vec(eng_, size=100, window=5, min_count=1, workers=48, iter=10)
# w2v_hindi = Word2Vec(hindi_, size=100, window=5, min_count=1, workers=48, iter=10)
# w2v_eng.save("save/w2v_eng.model")
# w2v_hindi.save("save/w2v_hindi.model")
w2v_eng = Word2Vec.load("save/w2v_eng.model")
w2v_hindi = Word2Vec.load("save/w2v_hindi.model")
# model.most_similar('used')[:5]

In [72]:
def m_(d):
    return [float(i) for i in d]
def glove(path):
    return { i.split()[0]:i.split()[1:] for i in read_(path)}

In [73]:
glove_eng = glove('save/eng/vectors.txt')
glove_hindi = glove('save/hindi/vectors.txt')
len(glove_eng), len(glove_hindi)

(8074, 10354)

# Words matching

## Word 2 Vec

In [51]:
# English using Word2Vec
save_text = L1
while True:
    closest = {i:[] for i in save_text.split('\n')}
    n = 5
    a = 0
    for i in tqdm(save_text.split('\n')):
        try: 
            closest[i].append([i[0] for i in w2v_eng.most_similar(i.split(',')[0])][:n])
            a+=1
        except: closest[i].append([None])

    print(f"Number of words matched:- {a}")

    #Hindi using Word2Vec
    # closest = {}
    n = 5
    a = 0
    for i in tqdm(save_text.split('\n')):
        try: 
            closest[i].append([i[0] for i in w2v_hindi.most_similar(i.split(',')[1])][:n])
            a+=1
        except: closest[i].append([None])
    print(f"Number of words matched:- {a}")

    d = 0
    m = ''
    dummy_ = [i.split(',')[0] for i in save_text.split('\n')]
    for k,v in closest.items():
        if v[0][0] != None and v[1][0] !=None:
            for j in v[0]:
                if remove_punctuation(j.strip()) in dummy_: continue
                try:
                    if lex[remove_punctuation(j.strip())] in v[1]:
                        m += f"{remove_punctuation(j.strip())},{lex[remove_punctuation(j.strip())]},{k.split(',')[-1]}\n"
                        print(v[1],remove_punctuation(j.strip()),lex[remove_punctuation(j.strip())])
                        d += 1
                except: pass

    print(f"Words added: {d}")
    save_text += m
    if d == 0: break

<ipython-input-51-fc35c83a6ca5>:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  closest[i].append([i[0] for i in w2v_eng.most_similar(i.split(',')[0])][:n])



Number of words matched:- 653


<ipython-input-51-fc35c83a6ca5>:21: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  closest[i].append([i[0] for i in w2v_hindi.most_similar(i.split(',')[1])][:n])



Number of words matched:- 662
['स्थल', 'जीवन', 'तुरंत', 'स्टेशन', 'karbonn'] life जीवन
['बड़ा', 'जितना', 'होना', 'रहती', 'सरल'] big बड़ा
['काफी', 'अच्छी', 'बहुत', 'बेहतर', 'क्वालिटी'] very बहुत
['बड़ा', 'देरी', 'स्पीड', 'जैसा', 'प्रदर्शन'] display प्रदर्शन
['समझदारी', 'बदला', 'नहीं', 'कहा', 'पाएगा'] wont नहीं
Words added: 5



Number of words matched:- 658



Number of words matched:- 667
Words added: 0


In [52]:
with open("save/w2v.json", "w") as f:
    f.write(json.dumps(closest))

with open("save/w2v.L1.txt", "w") as f:
    f.write(save_text)
with open("save/w2v.json", "r") as f:
    out = json.loads(f.read())
# out

## Glove

In [78]:
def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

# v1,v2 = [3, 45, 7, 2], [2, 54, 13, 15]
# cosine_similarity(v1,v2)

#English using Glove
# save_text = L1
while True:
    closest = {i:[] for i in save_text.split('\n')}
    n = 5
    a = 0
    for i in tqdm(save_text.split('\n')):
        word = i.split(',')[0]
        if word in glove_eng.keys():
            dummy = {}
            for j in glove_eng.keys():
                score = cosine_similarity([float(i) for i in glove_eng[j][-100:]],[float(i) for i in glove_eng[word][-100:]])
                dummy[score] = j
            closest[i].append([i[-1] for i in sorted(dummy.items(),reverse=True)][1:n+1])
            a+=1
        else: closest[i].append([None])
    print(f"Number of words matched:- {a}")

    #Hindi using Glove
    # closest = {}
    n = 5
    a = 0
    for i in tqdm(save_text.split('\n')):
        try: word = i.split(',')[1]
        except: continue
        if word in glove_hindi.keys():
            dummy = {}
            for j in glove_hindi.keys():
                score = cosine_similarity([float(i) for i in glove_hindi[j][-100:]],[float(i) for i in glove_hindi[word][-100:]])
                dummy[score] = j
            closest[i].append([i[-1] for i in sorted(dummy.items(),reverse=True)][1:n+1])
            a+=1
        else: closest[i].append([None])
    print(f"Number of words matched:- {a}")

    d = 0
    m = ''
    dummy_ = [i.split(',')[0] for i in save_text.split('\n')]
    for k,v in closest.items():
        if v[0][0] != None and v[1][0] !=None:
            for j in v[0]:
                if remove_punctuation(j.strip()) in dummy_: continue
                try:
                    if lex[remove_punctuation(j.strip())] in v[1]:
                        m += f"{remove_punctuation(j.strip())},{lex[remove_punctuation(j.strip())]},{k.split(',')[-1]}\n"
                        print(v[1],remove_punctuation(j.strip()),lex[remove_punctuation(j.strip())])
                        d += 1
                except: pass

    print(f"Words added: {d}")
    save_text += m
    if d == 0: break


Number of words matched:- 655



Number of words matched:- 664


In [ ]:
with open("save/glove.json", "w") as f:
    f.write(json.dumps(closest))

with open("save/glove.L1.txt", "w") as f:
    f.write(save_text)
    
with open("save/glove.json", "r") as f:
    out = json.loads(f.read())
# out

## End